# Глобальные переменные
__`main_dir`__: рабочий каталог скрипта<br>
__`graph_dir`__: подкаталог, в который будут сохраняться графики (будет создан, если не существует)<br>
__`table_path`__: путь к таблице<br>
__`log_name`__: название лог-файла (перенаправление вывода из консоли в файл; сейчас отключено)<br>
__`res_fname`__: название, под которым будет сохраняться таблица с результатами

In [ ]:
main_dir <- file.path("D:", "Desktop", "even_dialects")
graph_dir <- "plots"
table_path <- file.path("D:", "Desktop", "wordmor_table.csv")
setwd(main_dir)
# log_name <- "log.txt"
res_fname <- "res_new_1.csv"

# Библиотеки
__`fs`__: _path_sanitize()_<br>
__`tidyverse`__: _%>%, ..._<br>
__`lme4`__: _glmer()_<br>
__`Hmisc`__: _binconf()_<br>
__`effects`__: _predictorEffect()_<br>
__`ggpubr`__: _ggarrange()_

In [ ]:
library(fs)         # path_sanitize()
library(tidyverse)  # %>%, ...
library(lme4)       # glmer()
library(Hmisc)      # binconf()
library(effects)    # predictorEffect()
library(ggpubr)     # ggarrange()

# Функции для работы с морфемами
__`get_pos`__ → `character`: получить часть речи, записанную в названии столбца морфемы<br>
　`morpheme`: название столбца<br>

__`get_pos`__ → `data.frame/tibble`: получить часть речи, записанную в названии столбца морфемы<br>
　`src`: исходная таблица<br>
　`morpheme`: название столбца<br>

In [ ]:
get_pos <- function(morpheme) {
  return(sapply(strsplit(as.character(morpheme), "-"), tail, 1))
}

morpheme_features <- function(src, morpheme){
  src[, c(as.character(morpheme), "speaker", "corpus", "pos")] %>%
    filter(pos == get_pos(morpheme)) -> ret
  colnames(ret)[1] <- "cur_morpheme"
  return(ret)
}

# Класс переменной-обертки с информацией о регрессии `Fit_info`
__поля__ (к ним обращаться не нужно):<br>
　__`code_`__: код, с которым завершилась регрессия<br>
　__`message_`__: сообщение, с которым завершилась регрессия<br>
　__`pval_`__: p-value корпуса как фиксированного фактора<br>

__методы__:<br>
　__`is.err`__ → `TRUE/FALSE`: носители одного из диалектов не употребили морфему ни в одном случае<br>
　　　(невозможно посчитать ни p-value, ни доверительные интервалы)<br>
　__`converged`__ → `TRUE/FALSE`: регрессия успешно завершила работу (т.е. не возникло ошибок и она сошлась)<br>
　　　(если и __`is.err`__, __`converged`__ возвращают `FALSE`, значит, регрессия проработала без ошибок, но не сошлась -- тогда посчитать p-value и доверительные интервалы все равно можно)<br>
　__`p.val`__ → `double/NA`: получение значения p-value (когда возможно)<br>
　__`get.info`__ → `list`: получение кода и сообщения, например, для записи в лог или обработки ошибок
 
 __`get_fit_info`__ → `Fit_info`: конструирование переменной класса `Fit_info` с информацией о регрессии<br>
 　`fit`: регрессия

In [ ]:
Fit_info <- setRefClass(
  "Fit_info",
  fields =
    list(code_ = "numeric", message_ = "character", pval_ = "numeric"),
  methods =
    list(
      is.err = function() {
          return(pval_ == -1)
        },
      converged = function() {
          return(code_ == 0)
        },
      p.val = function() {
          return(ifelse(pval_ == -1, NA, pval_))
        },
      get.info = function() {
        return(list(code_, message_))
      }
    )
)

get_fit_info <- function(fit) {
  pv <- -1
  tryCatch({
      pv <- unname(coef(summary(fit))[2,4])
    },
      error = function(e){
        print(paste("Can't extract p-value:", e))
      }
    )
  
  return(Fit_info$new(code_ =
                        ifelse(
                          is.null(fit@optinfo$conv$lme4$code),
                          0,
                          fit@optinfo$conv$lme4$code),
                      message_ =
                        ifelse(
                          is.null(fit@optinfo$conv$lme4$messages),
                          "",
                          paste(fit@optinfo$conv$lme4$messages)),
                      pval_ = pv))
}

# Загрузка и первичная обработка таблицы
`read.csv` неправильно считывает заголовки столбцов (даже когда указана кодировка UTF-8), поэтому их приходится загружать отдельно

In [ ]:
data <- read.csv(table_path)
data <- data.frame(data)

con <- file(table_path, encoding = "UTF-8")
titles <- readLines(con, n = 1)
close(con)

colnames(data) <- strsplit(x = titles, split = '\\,')[[1]]

# Обработка таблицы
__`ignore_speakers`__: носители, которых нужно исключить из рассмотрения<br>

1. Исключение носителей
2. Исключение морфем с некорректными/неопределенными частями речи
3. Исключение не употребляющихся морфем и морфем, все записи о которых (0 и 1) относятся лишь к одному диалекту

__`morphemes`__: все рассматриваемые морфемы<br>
__`parts_of_speech`__: все рассматриваемые части речи<br>
__`speakers`__: все рассматриваемые носители + диалекты, к которым они принадлежат<br>

In [ ]:
ignore_speakers <- c('AL', 'NA', 'BP', 'children', 'VAK', 'NEN', 'T', '?', 'rh', 'TPA', 'R', 'LNZ', 'IAS')

data %>%
  filter(!(speaker %in% ignore_speakers)) -> data

# исключение морфем с неясной частью речи
colnames(data) -> morphemes
morphemes[!(morphemes %in% c("corpus", "speaker", "pos"))] -> morphemes

data$pos %>%
  unique() %>%
  droplevels() -> parts_of_speech

# исключение столбцов, в которых не размечена часть информации (присутствуют "?" или "*"), и столбцов с некорректно указанной частью речи
ignore_cols <- c()
for(cn in morphemes){
  if(grepl("?", cn, fixed = TRUE) | grepl("*", cn, fixed = TRUE) | !(sapply(strsplit(cn, "-"), tail, 1) %in% parts_of_speech)){
    ignore_cols <- c(ignore_cols, cn)
  }
}

data <- data[which(!(colnames(data) %in% ignore_cols))]
colnames(data) -> morphemes
morphemes[!(morphemes %in% c("corpus", "speaker", "pos"))] -> morphemes

# исключение не употребляющихся морфем и морфем, вхождения (0 и 1) которых относятся только к 1 диалекту
ignore_cols <- c()
for(morpheme in morphemes) {
  data[, c(as.character(morpheme), "corpus", "pos")] %>%
    filter(pos == sapply(strsplit(as.character(morpheme), "-"), tail, 1)) %>%
    count(corpus) %>%
    nrow() -> usage_corpus_wise
  data[, c(as.character(morpheme), "pos")] %>%
    filter(pos == sapply(strsplit(as.character(morpheme), "-"), tail, 1)) %>%
    dplyr::select(as.character(morpheme)) %>%
    sum() -> total_usage
  if(!((usage_corpus_wise == 2) & (total_usage > 0))) {
    ignore_cols <- c(ignore_cols, morpheme)
  }
}
data <- data[which(!(colnames(data) %in% ignore_cols))]
colnames(data) -> morphemes
morphemes[!(morphemes %in% c("corpus", "speaker", "pos"))] -> morphemes

# исключение частей речи
data %>%
  filter(!(pos %in% c('?'))) -> data

data$pos <- droplevels(data$pos)

data$pos %>%
  unique() -> parts_of_speech
  
data %>%
  dplyr::select(speaker, corpus) %>%
  unique() -> speakers

# Создание таблицы с результатами
- __`morpheme`__: название морфемы
- __`pos`__: часть речи, к которой принадлежит морфема
- __`rel_freq_k`__: относительная частота употребления морфемы в диалекте k
- __`abs_freq_k`__: абсолютная частота употребления морфемы в диалекте k
- __`rel_freq_s`__: относительная частота употребления морфемы в диалекте s
- __`abs_freq_s`__: абсолютная частота употребления морфемы в диалекте s

_Абсолютная частота_ -- количество `1` в столбце<br>
_Относительная частота_ -- отношение количества `1` в столбце к высоте столбца<br>
(с учетом диалекта)

In [ ]:
res_cols <- c("morpheme",  "pos", "rel_freq_k", "abs_freq_k", "rel_freq_s", "abs_freq_s")
res <- data.frame(matrix(ncol = length(res_cols), nrow = 0))
colnames(res) <- res_cols

# Первичное наполнение таблицы с результатами

In [ ]:
for(morpheme in morphemes){
  mor.data <- morpheme_features(data, morpheme)
  
  freq <- data.frame(matrix(ncol = 1, nrow = 2))
  colnames(freq) <- c("corpus")
  freq$corpus <- c("k", "s")
  freq$corpus <- as.factor(freq$corpus)
  
  mor.data %>%
    count(corpus) %>%
    rename(tot = n) %>%
    left_join(freq, ., by = "corpus") -> freq
  
  mor.data %>%
    filter(cur_morpheme == "1") %>%
    count(corpus) %>%
    rename(used = n) %>%
    left_join(freq, ., by = "corpus") -> freq
      
  freq %>%
    mutate(tot = ifelse(is.na(tot), 0, tot),
           used = ifelse(is.na(used), 0, used),
           rel_freq = ifelse(tot != 0, used / tot, 0)) -> freq
  
  res <- rbind(res, data.frame(morpheme =
                                 morpheme,
                               pos =
                                 get_pos(morpheme),
                               rel_freq_k =
                                 freq[freq$corpus == "k", ]$rel_freq,
                               abs_freq_k =
                                 freq[freq$corpus == "k", ]$used,
                               rel_freq_s =
                                 freq[freq$corpus == "s", ]$rel_freq,
                               abs_freq_s =
                                 freq[freq$corpus == "s", ]$used))
}

# Добавление в таблицу с результатами остальных столбцов
- __`pval_q_n`__: p-value регрессии `1`
- __`conv_q_n`__: сходимость регрессии `1`
- __`pval_r`__: p-value регрессии `2`
- __`conv_r`__: сходимость регрессии `2`
- __`mw`__: p-value теста Манна-Уитни
- __`loglik_q_n`__: log likelihood регрессии `1`
- __`loglik_r`__: log likelihood регрессии `2`
- __`ci_q_n_k_upper`__: верхняя граница дов. инт. регрессии `1` по диалекту `k`
- __`ci_q_n_k_lower`__: нижняя граница дов. инт. регрессии `1` по диалекту `k`
- __`ci_q_n_s_upper`__: верхняя граница дов. инт. регрессии `1` по диалекту `s`
- __`ci_q_n_s_lower`__: нижняя граница дов. инт. регрессии `1` по диалекту `s`
- __`ci_r_k_upper`__: верхняя граница дов. инт. регрессии `2` по диалекту `k`
- __`ci_r_k_lower`__: нижняя граница дов. инт. регрессии `2` по диалекту `k`
- __`ci_r_s_upper`__: верхняя граница дов. инт. регрессии `2` по диалекту `s`
- __`ci_r_s_lower`__: нижняя граница дов. инт. регрессии `2` по диалекту `s`

Регрессия `1`: __`morpheme ~ corpus + (corpus|speaker)`__<br>
Регрессия `2`: __`morpheme ~ corpus + (1|speaker)`__

In [ ]:
res %>%
  mutate(pval_q_n = NA,
         conv_q_n = NA,
         pval_r = NA,
         conv_r = NA,
         mw = NA,
         loglik_q_n = NA,
         loglik_r = NA,
         ci_q_n_k_upper = NA,
         ci_q_n_k_lower = NA,
         ci_q_n_s_upper = NA,
         ci_q_n_s_lower = NA,
         ci_r_k_upper = NA,
         ci_r_k_lower = NA,
         ci_r_s_upper = NA,
         ci_r_s_lower = NA) -> res

# Наполнение новых столбцов и генерация графиков
__`r_suff_q_n`__: суффикс для графиков с доверительными интервалами из регрессии `1`<br>
__`r_suff_r`__: суффикс для графиков с доверительными интервалами из регрессии `2`

In [ ]:
dir.create(graph_dir)
setwd(graph_dir)

gr_suff_q_n <- ".quasi_nested.png"
gr_suff_r <- ".random.png"

for(i in 1:nrow(res)) {
  
  print(paste(i, '/', nrow(res), ":", res[i, ]$morpheme))
  
  mor.data <- morpheme_features(data, res[i, ]$morpheme)
  
  # факторизация
  mor.data[, 1] <- as.factor(mor.data[, 1])
  
  mor.data %>%
    count(speaker, cur_morpheme) %>%
    spread(cur_morpheme, n) -> plot_data.unfiltered
  
  plot_data.unfiltered %>%
    mutate(`0` = ifelse(is.na(`0`), 0, `0`),
           `1` = ifelse(is.na(`1`), 0, `1`),
           tot_times = `1` + `0`,
           freq = `1` / tot_times) %>%
    inner_join(., speakers, by = "speaker") %>%
    rename(times_used = `1`) %>%
    dplyr::select(speaker, times_used, tot_times, freq, corpus) -> plot_data.unfiltered
  
  plot_data.unfiltered %>%
    filter(freq > 0) %>%
    arrange(desc(freq)) -> plot_data

  plot_data %>%
    mutate(num = seq(1000, 1000 + nrow(plot_data) - 1, 1),
           speaker_temp = speaker) %>%
    unite("speaker_plot", num, speaker_temp) -> plot_data

  # - - - регрессия - - -
  
  # масштабирование данных к отрезку [0; 1]
  mor.data %>%
    mutate(corpus.regr = (as.numeric(corpus) - 1),
           speaker.regr = (as.numeric(speaker) - 1) / summary(as.numeric(speaker))["Max."]) -> mor.data
  
  fit.quasi_nested.f_info <- NA
  fit.quasi_nested <- glmer(cur_morpheme ~ corpus + (corpus|speaker), data = mor.data, family = binomial,
                           control = glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 10000)))
  fit.quasi_nested.f_info <- get_fit_info(fit.quasi_nested)
  res[i, "pval_q_n"] <- fit.quasi_nested.f_info$p.val()
  res[i, "conv_q_n"] <- as.numeric(fit.quasi_nested.f_info$converged())
  
  
  fit.random.f_info <- NA
  fit.random <- glmer(cur_morpheme ~ corpus + (1|speaker), data = mor.data, family = binomial,
                           control = glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 10000)))
  fit.random.f_info <- get_fit_info(fit.random)
  res[i, "pval_r"] <- fit.random.f_info$p.val()
  res[i, "conv_r"] <- as.numeric(fit.random.f_info$converged())
  
    
  # - - - доверительные интервалы: биномиальное распределение - - -
  ci_cols <- c("speaker", "ci_l", "ci_h")
  ci <- data.frame(matrix(ncol = length(ci_cols), nrow = 0))
  colnames(ci) <- ci_cols

  for(j in seq(1, nrow(plot_data), 1)) {
    temp <- binconf(as.numeric(plot_data[j, "times_used"]), as.numeric(plot_data[j, "tot_times"]))
    ci <- rbind(ci, data.frame(speaker = plot_data[j, "speaker"], ci_l = temp[1, 2], ci_h = temp[1,3]))
  }
  
  tibble::enframe(seq(1, nrow(plot_data), 1)) %>%
    rename(X = value) %>%
    mutate(Y1 = ci$ci_l,
           Y2 = ci$ci_h,
           speaker = plot_data$speaker) %>%
    gather(key = "grp", value ="Y", Y1, Y2) %>%
    mutate(grp = as.character(X)) %>%
    dplyr::select(X, speaker, grp, Y)-> lines.distr
  
  # - - - генерация графика - - -
  plot_data %>%
  left_join(., lines.distr, by = "speaker") %>%
    ggplot(.,
           aes(x = speaker_plot, y = freq * 0.5)) +
    geom_bar(stat="identity", aes(fill = factor(corpus))) +
    theme(axis.text.x = element_text(angle = 90),
          legend.position="none") +
    labs(x = "Информанты",
         y = "Частота употребления",
         title = "Употребление") +
    geom_line(aes(speaker_plot, Y, group = speaker), size = 2) +
    scale_x_discrete(labels = plot_data$speaker) +
    scale_fill_manual(values = c("k" = "#f8766d", "s" = "#00bfc4"), aesthetics = "fill") -> plot.fit.hist
  
  
  # - - - доверительные интервалы: квази-вложенность - - -
  if(!fit.quasi_nested.f_info$is.err()) {
  
    ci_temp <- summary(predictorEffect("corpus", fit.quasi_nested))
    
    data.frame(ci_temp$lower, ci_temp$upper) %>%
      mutate(corpus = c("k", "s")) %>%
      rename(lower = ci_temp.lower,
             upper = ci_temp.upper) %>%
      as_tibble() -> fit.quasi_nested.ci
    
    res[i, "ci_q_n_k_lower"] <- fit.quasi_nested.ci[fit.quasi_nested.ci$corpus == "k",]$lower
    res[i, "ci_q_n_k_upper"] <- fit.quasi_nested.ci[fit.quasi_nested.ci$corpus == "k",]$upper
    res[i, "ci_q_n_s_lower"] <- fit.quasi_nested.ci[fit.quasi_nested.ci$corpus == "s",]$lower
    res[i, "ci_q_n_s_upper"] <- fit.quasi_nested.ci[fit.quasi_nested.ci$corpus == "s",]$upper
    
    # - - - генерация графика - - -
    fit.quasi_nested.ci %>%
      mutate(grp = as.numeric(corpus)) %>%
      gather(key = "grp", value = "Y", upper, lower) %>%
      inner_join(., plot_data.unfiltered, by = "corpus") %>%
      mutate(freq = replace(freq, duplicated(speaker), NA)) %>%
      ggplot(., aes(corpus, Y, group = corpus)) +
      labs(y = "Частота употребления", x = "Диалекты", title = "Доверительные интервалы по диалектам", color = "Диалекты:") +
      theme(axis.text.x = element_blank()) +
      geom_line(size = 2) +
      geom_point(aes(corpus, freq, color = corpus), size = 2, position = "jitter", na.rm = TRUE) +
      scale_x_discrete(labels = fit.quasi_nested.ci$corpus) -> plot.fit.quasi_nested.confints
  
    plot.fit.quasi_nested.joint <- annotate_figure(ggarrange(plot.fit.quasi_nested.confints, plot.fit.hist, ncol = 2, nrow = 1, widths = c(3,4),
                              common.legend = TRUE, legend = "bottom"),
              top = text_grob(paste("Морфема:", res[i, ]$morpheme), size = 14),
              bottom = text_grob(paste("Формула:", as.character(formula(fit.quasi_nested))[2],
                                       as.character(formula(fit.quasi_nested))[1],
                                       as.character(formula(fit.quasi_nested))[3]), size = 10))
  
    # - - - сохранение графика - - -
    ggsave(path_sanitize(paste0(i, gr_suff_q_n)), plot = plot.fit.quasi_nested.joint, width = 10, height = 4)
    file.rename(list.files(pattern = paste0(i, gr_suff_q_n)), paste0(as.character(res[i,]$morpheme), gr_suff_q_n))
  }


  # - - - доверительные интервалы: рандом-фактор - - -
  if(!fit.random.f_info$is.err()) {
  
    ci_temp <- summary(predictorEffect("corpus", fit.random))
    
    data.frame(ci_temp$lower, ci_temp$upper) %>%
      mutate(corpus = c("k", "s")) %>%
      rename(lower = ci_temp.lower,
             upper = ci_temp.upper) %>%
      as_tibble() -> fit.random.ci
    
    res[i, "ci_r_k_lower"] <- fit.random.ci[fit.random.ci$corpus == "k",]$lower
    res[i, "ci_r_k_upper"] <- fit.random.ci[fit.random.ci$corpus == "k",]$upper
    res[i, "ci_r_s_lower"] <- fit.random.ci[fit.random.ci$corpus == "s",]$lower
    res[i, "ci_r_s_upper"] <- fit.random.ci[fit.random.ci$corpus == "s",]$upper

    # - - - генерация графика - - -
    fit.random.ci %>%
      mutate(grp = as.numeric(corpus)) %>%
      gather(key = "grp", value = "Y", upper, lower) %>%
      inner_join(., plot_data.unfiltered, by = "corpus") %>%
      mutate(freq = replace(freq, duplicated(speaker), NA)) %>%
      ggplot(., aes(corpus, Y, group = corpus)) +
      labs(y = "Частота употребления", x = "Диалекты", title = "Доверительные интервалы по диалектам", color = "Диалекты:") +
      theme(axis.text.x = element_blank()) +
      geom_line(size = 2) +
      geom_point(aes(corpus, freq, color = corpus), size = 2, position = "jitter", na.rm = TRUE) +
      scale_x_discrete(labels = fit.random.ci$corpus) -> plot.fit.random.confints
  
    plot.fit.random.joint <- annotate_figure(ggarrange(plot.fit.random.confints, plot.fit.hist, ncol = 2, nrow = 1, widths = c(3,4),
                              common.legend = TRUE, legend = "bottom"),
              top = text_grob(paste("Морфема:", res[i, ]$morpheme), size = 14),
              bottom = text_grob(paste("Формула:", as.character(formula(fit.random))[2],
                                       as.character(formula(fit.random))[1],
                                       as.character(formula(fit.random))[3]), size = 10))
  
    # - - - сохранение графика - - -
    ggsave(path_sanitize(paste0(i, gr_suff_r)), plot = plot.fit.random.joint, width = 10, height = 4)
    file.rename(list.files(pattern=paste0(i, gr_suff_r)), paste0(as.character(res[i,]$morpheme), gr_suff_r))
  }
  
  
  # - - - Mann-Whitney - - -
  res[i, "mw"] <- wilcox.test(plot_data.unfiltered[plot_data.unfiltered$corpus == 'k', ]$freq,
            plot_data.unfiltered[plot_data.unfiltered$corpus == 's', ]$freq)$p.value
    
  
  #- - - Log-likelihood - - -
  if (!fit.quasi_nested.f_info$is.err()) {
    res[i, "loglik_q_n"] <- summary(fit.quasi_nested)$logLik[1]
  } else {
    res[i, "loglik_q_n"] <- NA
  }
  
  if(!fit.random.f_info$is.err()) {
    res[i, "loglik_r"] <- summary(fit.random)$logLik[1]
  } else {
    res[i, "loglik_r"] <- NA
  }

  # запись в лог (раскомментировать)
  # capture.output(paste(res[i,]$morpheme, paste(fit.quasi_nested.f_info$get.info(), collapse = " "), sep = " (quasi-nested): "), file = log_name, append = TRUE)
  # capture.output(paste(res[i,]$morpheme, paste(fit.random.f_info$get.info(), collapse = " "), sep = " (random): "), file = log_name, append = TRUE)
}

setwd(main_dir)

# Запись таблицы с результатами в файл

In [ ]:
write.table(res, file = res_fname, row.names = FALSE, dec = ",", sep = ";", quote = FALSE, fileEncoding = 'UTF-8')